**Team members:**

CORBISIERO GERARDO

LONGOBARDI FRANCESCO

LONGOBARDI GIOVANNI

MONTILLO ANDREA

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras
import numpy as np
import cv2

from keras import backend as K
from keras.models import Model
from keras.layers import Dense, Input, Lambda, Concatenate, Flatten, Dropout, GlobalMaxPool2D, GlobalAvgPool2D
from keras.layers.pooling import MaxPooling2D,AveragePooling2D
!pip install imageai
from imageai.Detection import ObjectDetection


def build_finetune_model():
    h = 224
    w = 224
    input_shape=(h, w, 3)
    dropout = 0.25
    num_classes = 32

    base_model = keras.applications.mobilenet.MobileNet(weights='imagenet',
                                    include_top=False,
                                    input_shape=input_shape)
    
    x = base_model.output   
    x = AveragePooling2D((5, 5), name='avg_pool')(x)
    x = Flatten()(x)
    x = Dropout(dropout)(x)
    predictions = Dense(num_classes, activation='softmax', name='finalfc')(x)
    
    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    finetune_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return finetune_model


def siamese_model(finetune_model):
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = finetune_model

    for x in base_model.layers:
        x.trainable = True
    
    model2 = Model(base_model.input, base_model.layers[-5].output)    

    x1 = model2(input_1)
    x2 = model2(input_2)

    x1 = Concatenate(axis=-1)([GlobalMaxPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalMaxPool2D()(x2), GlobalAvgPool2D()(x2)])


    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    x = L1_layer([x1, x2])


    x = Dense(512, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.25)(x)
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.25)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)
    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer='adam')
    
    return model

Using TensorFlow backend.


     |████████████████████████████████| 184kB 2.8MB/s 


In [0]:

def compute_area(lista):
  dx = lista[2] - lista[0]
  dy = lista[3] - lista[1]
  return dx*dy


def detect(detector, image):
    returned, detections = detector.detectObjectsFromImage(input_type ="array", input_image=np.array(image), output_type="array", minimum_percentage_probability=50)

    maximum = 0
    car = None
    for det in detections:
      area = compute_area(det["box_points"])
      if (det["name"] == "car" or det["name"] == "truck") and area>maximum:
        maximum = area
        imdata=image.astype('uint8')
        imdata=np.flip(imdata,-1)

        x, y, x2, y2 = det["box_points"]
        car = imdata[y:y2, x:x2]

    if maximum < 200*150*0.2:
      car = None

    return car


def get_detector():
  detector = ObjectDetection()
  model_path = "/content/drive/My Drive/Consegna ML/YOLO_Weights.h5"
  detector.setModelTypeAsYOLOv3()
  detector.setModelPath(model_path)
  detector.loadModel()
  
  return detector


def get_model(flag_crop = True):
  finetune_model = build_finetune_model()
  model = siamese_model(finetune_model)

  if flag_crop:
    filepath = "/content/drive/My Drive/Consegna ML/Siamese_withDetection.h5"
  else: 
    filepath = "/content/drive/My Drive/Consegna ML/Siamese.h5"

  model.load_weights(filepath)
  return model


# Code to build the model and the detector, loading their weights 
detector = get_detector()
print("\nYolo Detector configuration completed.\n")

model = get_model()
model_without_crop = get_model(flag_crop = False)
print("\nModel configuration completed.\n")


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Yolo Detector configuration completed.

17227776/17225924 [==============================] - 2s 0us/step

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model configuration completed.



In [0]:
def predict(a, b):

  predictions = []

  for i in range(0,len(a)):
    imm1 = a[i]
    imm2 = b[i]
    
    image1 = detect(detector, imm1)
    image2 = detect(detector, imm2)

    if image1 is not None and image2 is not None:

      image1 = (image1[...,::-1])
      image2 = (image2[...,::-1])

      image1 = cv2.resize(image1, (224,224) ,0,0, cv2.INTER_LINEAR)
      image2 = cv2.resize(image2, (224,224) ,0,0, cv2.INTER_LINEAR)

      image1 = image1.astype(np.float32)
      image2 = image2.astype(np.float32)      

      image1 /= 255
      image2 /= 255

      image1 = np.array(image1)
      image2 = np.array(image2)

      tmp = model.predict([[image1],[image2]])     
    
    else:       
      image1 = cv2.resize(imm1, (224,224) ,0,0, cv2.INTER_LINEAR)
      image2 = cv2.resize(imm2, (224,224) ,0,0, cv2.INTER_LINEAR)

      image1 = image1.astype(np.float32)
      image2 = image2.astype(np.float32)

      image1 /= 255
      image2 /= 255

      image1 = np.array(image1)
      image2 = np.array(image2)

      tmp = model_without_crop.predict([[image1],[image2]])
    
    predictions.append([round(tmp[0][0])])

  return np.array(predictions)
